In [7]:
# taken partially from tutorial at: https://stmorse.github.io/journal/espn-fantasy-python.html
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

league1 = 380134
league2 = 1553391
leagues=[league1, league2]

weeks = 3

scores={}

for i in range(len(leagues)):
    scores[i] = {}
for week in range(1, weeks+1):
    for league in range(len(leagues)):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard', 
                         params={'leagueId': leagues[league], 'seasonId': 2018, 'matchupPeriodId': week})
        scores[league][week] = r.json()

df = []
# test = pd.DataFrame.from_dict(scores[1], orient='index')
# test.reset_index(level=0, inplace=True)
# display(test)
# print(scores[1])
for league in range(len(leagues)):
    for key in scores[league]:
        temp = scores[league][key]['scoreboard']['matchups']
        for match in temp:
#           create ints to determine who won
            home = 0
            away = 0
            if match["winner"]=="home":
                home=1
                away=-1
            else:
                home=-1
                away=1
                
            df.append([key, 
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score'],
                       home,
                       away
                      ])
df = pd.DataFrame(df, columns=['Week', 'HomeAbbrev', 'AwayAbbrev', 'HomeScore', 'AwayScore', 'Home Record', 'Away Record'])
df.head()
df = (df[['Week', 'HomeAbbrev', 'HomeScore', 'Home Record']]
      .rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score', 'Home Record':'# Wins - # Losses'})
      .append(df[['Week', 'AwayAbbrev', 'AwayScore', 'Away Record']]
             .rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score', 'Away Record' : '# Wins - # Losses'}))
     )
display(df)
df1 = df.groupby("Abbrev")["Score"].describe().drop("count", axis=1)
s2 = df.groupby("Abbrev")['# Wins - # Losses'].sum()
s1 = df.groupby("Abbrev")['Score'].sum()
finalDF = df1.join(s2).join(s1).rename(columns={'Score':"Total Score"})
#  df.sort_values(by=['col1', 'col2'])
finalDF = finalDF.sort_values(by=['# Wins - # Losses','Total Score'], ascending=False)
finalDF = finalDF.reset_index()
finalDF.index +=1
display(finalDF)

,Week,Abbrev,Score,# Wins - # Losses
0,1,KANG,115.34,-1
1,1,LEE,102.76,1
2,1,SRIV,71.72,-1
3,1,HARN,120.64,1
4,1,FITZ,107.34,1
5,1,ROCK,111.44,1
6,1,LARS,111.56,1
7,2,BAIL,102.28,-1
8,2,TRAN,60.18,-1
9,2,Cam,110.20,1


,Abbrev,mean,std,min,25%,50%,75%,max,# Wins - # Losses,Total Score
1,PETR,136.273333,44.290967,98.82,111.83,124.84,155.00,185.16,3,408.82
2,HARN,117.040000,6.850547,109.14,114.89,120.64,120.99,121.34,3,351.12
3,FITZ,110.413333,3.299475,107.34,108.67,110.00,111.95,113.90,3,331.24
4,NATH,85.080000,14.840310,75.34,76.54,77.74,89.95,102.16,3,255.24
5,LURI,115.146667,12.809143,100.36,111.30,122.24,122.54,122.84,1,345.44
6,LEE,114.753333,13.289249,102.76,107.61,112.46,120.75,129.04,1,344.26
7,DREW,112.880000,19.950018,90.00,106.00,122.00,124.32,126.64,1,338.64
8,AUTO,107.340000,29.236751,73.80,97.29,120.78,124.11,127.44,1,322.02
9,ROCK,98.280000,13.140046,85.16,91.70,98.24,104.84,111.44,1,294.84
10,B50,95.513333,10.821762,83.04,92.07,101.10,101.75,102.40,1,286.54
